# Few-shot Machine Translation + Cross-lingual Transfer

**Cross-lingual transfer** is an effective strategy in multilingual NLP where we leverage a high-resource language pair (like Hindi-English) to improve performance on a **low-resource language pair** (like Kannada-English).

This two-phase fine-tuning setup works well with multilingual models like **mBART**, enabling better generalization when only limited parallel data is available for the target language.

---

### What is Cross-lingual Transfer?

* **Pre-train on High-Resource Pair**: Fine-tune a multilingual model on a large dataset like Hindi ↔ English.
* **Few-shot Transfer to Low-Resource Pair**: Further fine-tune the same model on a smaller dataset for Kannada ↔ English.

In [ ]:
!pip install transformers torch datasets accelerate --quiet

In [39]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

### 🔹 Step 1: High-Resource Fine-tuning (Hindi ↔ English)

We fine-tune on a high-resource language pair to initialize cross-lingual knowledge.

In [ ]:
# For demo, let's use 'wmt14' dataset, or replace with your dataset
high_resource_dataset = load_dataset("wmt14", "hi-en")

In [47]:
high_resource_dataset['train'][0]

{'translation': {'en': 'January 0', 'hi': '० जनवरी'}}

#### Preprocessing

In [ ]:
def preprocess_function(examples, src_lang, tgt_lang):
    # Set tokenizer source and target language codes correctly
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang

    inputs = [ex[src_lang] for ex in examples["translation"]]
    targets = [ex[tgt_lang] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#### Tokenize Dataset

In [ ]:
# Prepare datasets for fine-tuning on Hindi-English
tokenized_high_resource = high_resource_dataset["train"].map(
    lambda x: preprocess_function(x, "hi_IN", "en_XX"),
    batched=True,
    remove_columns=high_resource_dataset["train"].column_names,
)

#### Train

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./hi-en-mbart-finetuned",
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    save_total_limit=1,
    save_steps=500,
    logging_steps=100,
    learning_rate=3e-5,
    weight_decay=0.01,
    report_to="none",
    disable_tqdm=False,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_high_resource,
)

In [ ]:
# Fine-tune on Hindi-English
trainer.train()

### 🔹 Step 2: Few-shot Fine-tuning on Kannada ↔ English

Now we adapt the model for the target low-resource pair using few-shot learning.


In [ ]:
few_shot_dataset = load_dataset("path_to_your_kn_en_dataset")  # Replace with your dataset

#### Preprocess Few-shot Dataset

In [ ]:

tokenized_few_shot = few_shot_dataset["train"].map(
    lambda x: preprocess_function(x, "kn", "en"),
    batched=True,
    remove_columns=few_shot_dataset["train"].column_names,
)

#### Few-shot Training


In [ ]:
# Update training arguments for few-shot
few_shot_args = TrainingArguments(
    output_dir="./kn-en-mbart-fewshot",
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",
    num_train_epochs=5,
    save_total_limit=2,
    save_steps=500,
    logging_steps=100,
    learning_rate=3e-5,
    weight_decay=0.01,
    report_to="none",
    disable_tqdm=False,
    fp16=True,
)

few_shot_trainer = Trainer(
    model=model,
    args=few_shot_args,
    train_dataset=tokenized_few_shot,
)

In [ ]:
# Few-shot fine-tuning
few_shot_trainer.train()

### Summary

| Phase                  | Language Pair     | Dataset Size      | Purpose                               |
| ---------------------- | ----------------- | ----------------- | ------------------------------------- |
| High-resource Pretrain | Hindi ↔ English   | Large (WMT)       | Transfer learning initialization      |
| Few-shot Fine-tune     | Kannada ↔ English | Few-shot (500–2K) | Domain adaptation for low-resource MT |

This approach allows better generalization and transfer of knowledge to under-resourced translation tasks.
